# Output 문장 구조 EDA

In [24]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
def make_dataframe(path: str) -> pd.DataFrame:
    """
    Read a json file and return a pandas DataFrame.

    Parameters:
    path (str): Path to the json file.

    Returns:
    pd.DataFrame: DataFrame of the json file.
    """
    # Read the json file
    with open(path, 'r') as file:
        data = json.load(file)

    # Create a DataFrame
    # columns = ['id', 'conversation', 'subject_keyword', 'output']
    df = pd.DataFrame(data)
    df['conversation'] = df['input'].apply(lambda x: x['conversation'])
    df['subject_keyword'] = df['input'].apply(lambda x: x['subject_keyword'])

    # Drop the 'input' column
    df.drop('input', axis=1, inplace=True)

    # Speakers in the conversation
    df['speakers'] = df['conversation'].apply(lambda turns: list(set(turn['speaker'] for turn in turns)))

    # Reorder the columns
    df = df[['id', 'conversation', 'subject_keyword', 'speakers', 'output']]

    return df

In [26]:
train_df = make_dataframe('../resource/data/일상대화요약_train.json')
dev_df = make_dataframe('../resource/data/일상대화요약_dev.json')
test_df = make_dataframe('../resource/data/일상대화요약_test.json')

## output의 구문 구조의 정성적 평가 결과에 따른 정량적 평가


### 정성적 평가 결과output은 총 3가지 구조로 이루어져 있음

- 맨 처음에 **전반적인 요약** 이 나오고
- 두 번째로 **speaker 1**이 한 말에 대한 요약이 나오고
- 세 번째로 **speaker 2**가 한 말에 대한 요약으로 마무리된다

<br/>

- 실제로 모든 데이터가 위와 같은 구조로 이루어져 있는지를 통계로 확인


    - **문장 안에 speaker1과 speaker2가 모두 존재하는지 확인**

        - 모두 존재함!

    <br/>

    - **그 순서가 speaker1, speaker2 순서로 이루어져 있는지 확인**

        - 모두 그렇지는 않음! 
        - train dataset의 경우 506개 중에 328개
            
    <br/>
            
    - **그렇다면, conversation의 시작하는 speaker가 맨 앞에 오고, 그 다음 speaker가 뒤에 오는 순서인가?**

        - 위의 sample과 이어지는 다음 sample인 경우 speaker2가 먼저 conversation을 시작하기도 함
        - train dataset의 경우 492 / 506, dev dataset의 경우 100 / 102
            

### 결론
- 일단 conversation을 시작하는 speaker가 첫번째로 요약되고, 그 이후 그 다음 speaker가 요약되는 경우가 압도적으로 많았다

    - train의 경우 492/506, dev의 경우 100/102
    - 이를 구문 구조의 output이 나오도록 강제시켜주는 것이 일반화에 도움을 줄 수 있어보인다

In [80]:
from copy import deepcopy

def output_semantic_structure_statistics(df: pd.DataFrame) -> None:
    """
    Print the statistics of the semantic structure of the output.

    Parameters:
    df (pd.DataFrame): DataFrame of the data.
    """
    # Copy the DataFrame
    df = deepcopy(df)


    # Check the speakers in the output
    speakers_in_output = df.apply(lambda row: not ((row['speakers'][0]) in row['output'] and (row['speakers'][1] in row['output'])), axis=1)
    print('The number of samples that Speakers are not in the output:', speakers_in_output.sum())


    # Split outputs into sentences by '.'
    sentences = df['output'].apply(lambda x: x.split('.'))


    # join the sentences without a first sentence
    df['output'] = sentences.apply(lambda x: ' '.join(x[1:]))


    # sort the speakers name
    df['speakers'] = df['speakers'].apply(lambda x: sorted(x))


    # Check the first indexes of the two speakers and the speaker1 - speaker2 order in the output 
    correct_order_sample = df.apply(lambda row: row['output'].find(row['speakers'][0]) < row['output'].find(row['speakers'][1]), axis=1)
    print('The number of samples that the order of the speakers is (speaker1 - speaker2) :', correct_order_sample.sum(), '/', len(df))


    # check the starter speaker of the conversation is the first speaker in the output
    def compare_starter_speaker_is_first_speaker(row):
        starter_speaker = row['conversation'][0]['speaker']
        output = row['output']
        if row['output'].find(row['speakers'][0]) < row['output'].find(row['speakers'][1]):
            # When the order of the speakers is (speaker1 - speaker2)
            return True
        else:
            # When the order of the speakers is (speaker2 - speaker1)
            # Check the starter speaker is the second speaker
            # If the starter speaker is the second speaker, return True else False
            if starter_speaker == row['speakers'][1]:
                return True
            else:
                return False

    starter_speaker_is_first_speaker = df.apply(compare_starter_speaker_is_first_speaker, axis=1)
    print('The number of samples that the starter speaker is the first speaker in the output:', starter_speaker_is_first_speaker.sum(), '/', len(df))

    # Sample indexes that The starter speaker is not the first speaker in the output
    print('Sample indexes that The starter speaker is not the first speaker in the output:',
          df[~starter_speaker_is_first_speaker].index.tolist())

In [81]:
output_semantic_structure_statistics(train_df)

The number of samples that Speakers are not in the output: 0
The number of samples that the order of the speakers is (speaker1 - speaker2) : 328 / 506
The number of samples that the starter speaker is the first speaker in the output: 492 / 506
Sample indexes that The starter speaker is not the first speaker in the output: [31, 32, 33, 62, 96, 110, 141, 304, 305, 323, 328, 335, 336, 475]


In [82]:
output_semantic_structure_statistics(dev_df)

The number of samples that Speakers are not in the output: 0
The number of samples that the order of the speakers is (speaker1 - speaker2) : 73 / 102
The number of samples that the starter speaker is the first speaker in the output: 100 / 102
Sample indexes that The starter speaker is not the first speaker in the output: [81, 83]


<br/>

### speaker1의 요약 내용과, spekaer2의 요약 내용의 길이를 각각 확인 
    
- 각 인물의 발화 내용 길이와 요약 내용의 길이 간의 상관관계 파악
    - 만약 길이에 따라 늘어난다면, 실제 학습된 모델도 그러하는지 파악
    - 만약 길이가 항상 고정된다면, 그에 따른 prompt 추가 및 추가 아이디어 생각 

---

## Output 구조에 맞게 모델이 내뱉도록 하기 위해 할 수 있는 것들

- **prompt를 구성한다**
    - 예를 들어, "요약을 하는데 맨 첫 문장은 전반적인 대화에 대한 요약, 이어서 'speaker 1은 ~~' 형태의 여러개의 문장, 마지막으로 'speaker 2는 ~' 형태의 여러개의 문장으로 작성해줘'

</br>

- **대화 내역의 노이즈를 없앤 후 모델에 전달한다**
    - 대화 내역과 output 간의 상관관계를 파악하여 필요한 feature들과 불필요한 feautre들을 파악하고
    - 필요한 feature들만 남겨 모델의 입력으로 사용한다

</br>

- **최대 길이에 의해 짤리는 부분을 완전히 없앤다**
    - 모델이 맨 마지막 지시(instruction)에 해당하는 부분의 토큰까지 모두 입력으로 받아낼 수 있도록 
    - 최대 길이를 조절하던지, 앞선 방식으로 입력을 줄여서 사용한다

</br>

- **lost in the middle 문제가 발생하는지 파악하고 이를 해결한다**
    - 대화 내역이 워낙 길다 보니까, 중간에 해당하는 부분이 제대로 요약되고 있는지를 validation dataset에 대해 평가한다
    - 만약 제대로 요약되지 않고 있다면, 이를 해결하기 위해 
        - 앞서 분석한 요약에 별로 쓸모가 없는 문장들을 중간에 배치하고
        - 중요한 문장을 앞 뒤에 배치함으로써 해결한다

<br/>